<a href="https://colab.research.google.com/github/FadiKais1/Rocchio-Algorithm-Overview-and-summarization/blob/main/Rocchio_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Solution: Rocchio Algorithm — Full Step-by-Step Implementation,

- Overview

In this exercise we need to implement the Rocchio algorithm in order to compute an improved query vector q(opt).

We receive 50 documents, each already represented as Bag of Words with tf-idf scores.
We need to:

Build the vocabulary using sports-related words (team, coach, hockey, soccer, win, loss, season…).

Compute cosine similarity between each document and the query terms to label documents as relevant or non-relevant.

If similarity > 0 → relevant

If similarity = 0 → not relevant

Choose 20% most similar relevant documents and 80% most similar non-relevant documents.

Use the Rocchio formula: q(opt)​=μR+μ′R′−μ′′NR .

where:

R: average vector of relevant docs

R′: next relevant docs (top similarities)

NR: non-relevant docs


μ,μ
′
,μ
′′
 — weights.

Show output:

Top 5 most important words in q(opt)


Top 3 documents closest to q(opt)
	​


This is exactly what we will implement in this home work.

- Some Overview about the algohrithm used before showing the solution.

Summary of the Algorithm and Its Role in Computing Similarities:


The Rocchio algorithm is a classical method in information retrieval used to improve a query based on feedback from documents.
The main idea is that a query can be “moved” closer to relevant documents and further away from non-relevant ones.
This is done by representing both documents and queries as vectors in the same feature space, usually created using Bag of Words and tf-idf scores.

Because documents and queries become points in a high-dimensional numerical space, we can measure how similar they are using the cosine similarity metric.
Cosine similarity checks the angle between two vectors:

A small angle (cosine close to 1) means the document is similar to the query.

A cosine of 0 means they share no direction (no common meaning).

In this exercise, cosine similarity is used for two main purposes:

To decide which documents are relevant (high similarity) and non-relevant (low similarity).

To build the Rocchio vectors 𝑅, R′, and NR, which represent the average of each group.
Once the relevant and non-relevant groups are identified, Rocchio updates the query using the formula:

𝑞(𝑜𝑝𝑡)=𝜇𝑅+𝜇′𝑅′−𝜇′′𝑁𝑅 .

This generates a new optimized query vector that:

Increases weights for terms that appear often in relevant documents

Decreases weights for terms common in non-relevant documents

This updated query reflects the true “topic direction” discovered from the documents.
Finally, the similarity between this optimized query and all documents shows which documents best match the new query meaning.

Overall, this algorithm connects the ideas of vector representation, term weighting (tf-idf), cosine similarity, and relevance feedback into one coherent method for improving information retrieval performance.
In our implementation, we followed all these steps: computing similarities, building R/R′/NR, and constructing q(opt), then finding the documents closest to the optimized query.

We will simulate the document set that should match real 50 docs ,and
The structure matches the expected exercise.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------------------------
# Step 1: Vocabulary and Simulated Document Set
# ---------------------------------------------
vocab = ["team","coach","hockey","baseball","soccer",
         "penalty","score","win","loss","season"]

V = len(vocab)
N = 50   # number of docs

np.random.seed(1)
documents = np.random.rand(N, V) * 0.4   # simulate tf-idf-like values

# ---------------------------------------------
# Step 2: Build the Query Vector
# ---------------------------------------------
query = np.ones(V)
query = query / np.linalg.norm(query)

# ---------------------------------------------
# Step 3: Compute Cosine Similarity
# ---------------------------------------------
similarities = cosine_similarity(documents, query.reshape(1,-1)).flatten()

# Create a DataFrame for full view
df_docs = pd.DataFrame(documents, columns=vocab)
df_docs["cosine_similarity"] = similarities
df_docs["doc_id"] = range(N)

# Order columns nicely
df_docs = df_docs[["doc_id"] + vocab + ["cosine_similarity"]]

print("=== View 1: Full TF-IDF Matrix + Cosine Similarity ===")
display(df_docs.head(10))


# ---------------------------------------------
# Step 4: Select Top Relevant & Non-Relevant Docs
# ---------------------------------------------
# Sort by similarity descending
sorted_idx = np.argsort(similarities)[::-1]

# 20% relevant, 80% non-relevant
k_rel = max(1, int(0.2 * N))
k_non = max(1, int(0.8 * N))

top_relevant = sorted_idx[:k_rel]
top_non_relevant = sorted_idx[-k_non:]

df_rel = df_docs.loc[top_relevant]
df_non = df_docs.loc[top_non_relevant]

print("\n=== View 2: Top 20% Relevant Documents ===")
display(df_rel)

print("\n=== View 3: Bottom 80% Non-Relevant Documents ===")
display(df_non)


# ---------------------------------------------
# Step 5: Compute R, R' (strong relevant), and NR
# ---------------------------------------------
R = documents[top_relevant].mean(axis=0)

# Choose half of the relevant docs as "strong relevant" R'
half_rel = max(1, k_rel // 2)
R_prime = documents[top_relevant[:half_rel]].mean(axis=0)

NR = documents[top_non_relevant].mean(axis=0)

df_R = pd.DataFrame([R], columns=vocab)
df_Rp = pd.DataFrame([R_prime], columns=vocab)
df_NR = pd.DataFrame([NR], columns=vocab)

print("\n=== View 4: Vector R (mean of relevant docs) ===")
display(df_R)

print("\n=== View 5: Vector R' (strong relevant docs) ===")
display(df_Rp)

print("\n=== View 6: Vector NR (mean of non-relevant docs) ===")
display(df_NR)


# ---------------------------------------------
# Step 6: Rocchio Formula
# ---------------------------------------------
mu = 1.0
mu_p = 0.75
mu_n = 0.25

q_opt = mu*R + mu_p*R_prime - mu_n*NR

df_qopt = pd.DataFrame([q_opt], columns=vocab)

print("\n=== View 7: q_opt Vector (Rocchio Output) ===")
display(df_qopt)


# ---------------------------------------------
# Step 7: Top 5 terms in q_opt
# ---------------------------------------------
top5_idx = np.argsort(q_opt)[::-1][:5]

df_top5 = pd.DataFrame({
    "term": [vocab[i] for i in top5_idx],
    "weight": [q_opt[i] for i in top5_idx]
})

print("\n=== View 8: Top 5 Most Important Terms in q_opt ===")
display(df_top5)


# ---------------------------------------------
# Step 8: Documents Closest to q_opt
# ---------------------------------------------
q_opt_norm = q_opt / np.linalg.norm(q_opt)
doc_sim_to_qopt = cosine_similarity(documents, q_opt_norm.reshape(1,-1)).flatten()

closest3 = np.argsort(doc_sim_to_qopt)[::-1][:3]

df_closest = df_docs.loc[closest3]

print("\n=== View 9: Top 3 Documents Closest to q_opt ===")
display(df_closest)


=== View 1: Full TF-IDF Matrix + Cosine Similarity ===


,doc_id,team,coach,hockey,baseball,soccer,penalty,score,win,loss,season,cosine_similarity
0,0,0.166809,0.288130,0.000046,0.120933,0.058702,0.036935,0.074504,0.138224,0.158707,0.215527,0.836089
1,1,0.167678,0.274088,0.081781,0.351247,0.010955,0.268187,0.166922,0.223476,0.056155,0.079241,0.847760
2,2,0.320298,0.387305,0.125370,0.276929,0.350556,0.357843,0.034018,0.015622,0.067932,0.351257,0.849218
3,3,0.039339,0.168443,0.383156,0.213266,0.276751,0.126206,0.274600,0.333850,0.007315,0.300058,0.872777
4,4,0.395544,0.299266,0.112178,0.315712,0.041290,0.179157,0.363438,0.117446,0.115110,0.052011,0.846027
5,5,0.007747,0.271534,0.084651,0.106219,0.196629,0.021345,0.229647,0.058691,0.235722,0.279903,0.832219
6,6,0.040934,0.165622,0.277760,0.165672,0.019981,0.214359,0.265518,0.205956,0.377838,0.234622,0.888464
7,7,0.361361,0.054990,0.055711,0.322957,0.159071,0.066142,0.371003,0.139106,0.300325,0.290399,0.864331
8,8,0.353322,0.249469,0.300377,0.139559,0.107971,0.358354,0.171236,0.385936,0.265377,0.248678,0.943832
9,9,0.045898,0.379796,0.179965,0.231356,0.163255,0.094811,0.361352,0.229472,0.001148,0.246858,0.854610



=== View 2: Top 20% Relevant Documents ===


,doc_id,team,coach,hockey,baseball,soccer,penalty,score,win,loss,season,cosine_similarity
20,20,0.380070,0.222661,0.366243,0.256626,0.156003,0.194396,0.241724,0.219819,0.370473,0.367493,0.960389
26,26,0.300009,0.343326,0.302033,0.279223,0.345792,0.129072,0.268316,0.180350,0.152841,0.164325,0.953718
8,8,0.353322,0.249469,0.300377,0.139559,0.107971,0.358354,0.171236,0.385936,0.265377,0.248678,0.943832
28,28,0.351999,0.361537,0.265088,0.108083,0.100947,0.341959,0.211086,0.320864,0.228995,0.293257,0.943701
41,41,0.043608,0.253515,0.321185,0.278720,0.306485,0.136982,0.338341,0.171508,0.329604,0.250598,0.935489
35,35,0.308095,0.292691,0.103879,0.102828,0.252921,0.138119,0.318635,0.178458,0.313100,0.396189,0.926769
23,23,0.183952,0.218539,0.319441,0.114288,0.196101,0.239644,0.006213,0.237393,0.173471,0.322944,0.915139
27,27,0.160592,0.126954,0.248768,0.172099,0.389521,0.271120,0.079428,0.170680,0.137338,0.319056,0.914828
32,32,0.166339,0.246674,0.093466,0.040787,0.206343,0.190856,0.061069,0.248722,0.217604,0.261655,0.914498
49,49,0.206907,0.366562,0.170590,0.098958,0.148518,0.372744,0.374747,0.337732,0.368083,0.091160,0.910746



=== View 3: Bottom 80% Non-Relevant Documents ===


...

,team,coach,hockey,baseball,soccer,penalty,score,win,loss,season
0,0.419292,0.424718,0.436701,0.271464,0.326389,0.362216,0.340691,0.38563,0.408513,0.421581



=== View 8: Top 5 Most Important Terms in q_opt ===


,term,weight
0,hockey,0.436701
1,coach,0.424718
2,season,0.421581
3,team,0.419292
4,loss,0.408513



=== View 9: Top 3 Documents Closest to q_opt ===


,doc_id,team,coach,hockey,baseball,soccer,penalty,score,win,loss,season,cosine_similarity
20,20,0.380070,0.222661,0.366243,0.256626,0.156003,0.194396,0.241724,0.219819,0.370473,0.367493,0.960389
28,28,0.351999,0.361537,0.265088,0.108083,0.100947,0.341959,0.211086,0.320864,0.228995,0.293257,0.943701
8,8,0.353322,0.249469,0.300377,0.139559,0.107971,0.358354,0.171236,0.385936,0.265377,0.248678,0.943832
